In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Set 준비

In [2]:
train_data = pd.read_csv('train_set.csv', index_col = 0)
test_data = pd.read_csv('test_set.csv', index_col = 0)

In [3]:
train_data

,date,cabbage,radish,garlic,onion,daikon,cilantro,artichoke
0,2021-01-02,713.00,760.00,4182.76,1282.48,2163.33,3862.77,12340.35
1,2021-01-04,1946.90,708.50,4427.22,1381.92,2083.49,3539.80,8773.32
2,2021-01-05,1307.79,996.28,4731.48,1396.43,2316.91,3281.04,10148.78
3,2021-01-06,1942.60,1012.55,4420.99,1484.83,2555.18,3440.97,11108.13
4,2021-01-07,1872.97,694.47,4080.54,1488.01,2740.90,3018.87,11615.35
...,...,...,...,...,...,...,...,...
1241,2024-12-26,2548.80,1332.04,5217.14,1161.23,1487.55,11351.61,11099.88
1242,2024-12-27,3642.66,1545.93,4674.20,1200.78,1503.53,10328.21,11454.85
1243,2024-12-28,2941.62,1325.48,4317.21,1292.75,1396.15,12691.50,11325.36
1244,2024-12-30,2287.54,2270.71,4904.38,1167.73,1566.44,10164.39,11842.47


In [4]:
test_data

,date,cabbage,radish,garlic,onion,daikon,cilantro,artichoke
0,2025-01-02,5078.12,3229.76,9820.4,2056.0,3559.40,36314.89,29710.0
1,2025-01-03,5026.88,3205.71,9579.2,2046.4,3509.00,35636.17,29818.0
2,2025-01-06,5211.25,3330.00,9958.0,2029.2,3495.00,35021.28,29850.0
3,2025-01-07,5318.33,3400.73,9794.0,2030.8,3508.20,34848.94,29754.0
4,2025-01-08,5366.88,3415.12,9797.2,2026.8,3493.00,34891.49,29706.0
...,...,...,...,...,...,...,...,...
75,2025-04-24,5035.83,2958.18,10985.6,3168.6,2036.80,25229.79,28246.0
76,2025-04-25,5011.46,3008.48,11102.0,3212.8,2026.40,25034.04,28766.0
77,2025-04-28,4983.54,2949.39,11490.8,3162.4,2037.80,24889.36,28818.0
78,2025-04-29,4954.38,2931.52,11536.2,3143.6,2054.80,24710.64,28664.0


## EDA 진행

In [5]:
print(train_data.info())
print('=' * 50)
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1246 entries, 0 to 1245
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       1246 non-null   object 
 1   cabbage    1240 non-null   float64
 2   radish     1241 non-null   float64
 3   garlic     1239 non-null   float64
 4   onion      1239 non-null   float64
 5   daikon     1244 non-null   float64
 6   cilantro   1236 non-null   float64
 7   artichoke  1239 non-null   float64
dtypes: float64(7), object(1)
memory usage: 87.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 0 to 79
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       80 non-null     object 
 1   cabbage    80 non-null     float64
 2   radish     80 non-null     float64
 3   garlic     80 non-null     float64
 4   onion      78 non-null     float64
 5   daikon     80 non-null     float64
 6   cilantro 

In [6]:
print(train_data.isnull().sum())
print('=' * 50)
print(test_data.isnull().sum())

date          0
cabbage       6
radish        5
garlic        7
onion         7
daikon        2
cilantro     10
artichoke     7
dtype: int64
date         0
cabbage      0
radish       0
garlic       0
onion        2
daikon       0
cilantro     0
artichoke    0
dtype: int64


- Nan 값이 존재. 해당 값들을 해당 월의 평균 값으로 대체

In [7]:
train_data['date'] = pd.to_datetime(train_data['date'])
train_data['year_month'] = train_data['date'].dt.to_period('M')

test_data['date'] = pd.to_datetime(test_data['date'])
test_data['year_month'] = test_data['date'].dt.to_period('M')

rows = train_data.isna().any(axis=1)
print(train_data[rows].head())

          date  cabbage  radish   garlic   onion   daikon  cilantro  \
31  2021-02-07   553.20  356.68      NaN  1030.0  3772.44       NaN   
223 2021-09-20  2985.97  576.81  6959.62     NaN  1441.37   1706.55   
336 2022-02-03  1668.21  435.42  6796.20   450.0  1904.38       NaN   
411 2022-05-01      NaN     NaN  8100.00     NaN      NaN       NaN   
424 2022-05-15      NaN     NaN  5440.00     NaN      NaN       NaN   

     artichoke year_month  
31     7213.89    2021-02  
223    5121.82    2021-09  
336   11550.61    2022-02  
411        NaN    2022-05  
424        NaN    2022-05  


In [8]:
# test 데이터 속 cilantror != cilantro
test_data.columns = train_data.columns

features = train_data.drop(columns=['date','year_month']).columns

for feature in features:
    #------------train data--------------
    train_avg_month = train_data.groupby('year_month')[feature].mean().round(2)
    train_rows_NaN_feature = train_data[train_data[feature].isna()].index
    
    for idx in train_rows_NaN_feature:
        year_month = train_data.at[idx, 'year_month']
        if pd.notna(train_avg_month.get(year_month)):  
            train_data.at[idx, feature] = train_avg_month[year_month]
            
    #------------test data--------------
    test_avg_month = test_data.groupby('year_month')[feature].mean().round(2)
    test_rows_NaN_feature = test_data[test_data[feature].isna()].index
    
    for idx in test_rows_NaN_feature:
        year_month = test_data.at[idx, 'year_month']
        if pd.notna(test_avg_month.get(year_month)):  
            test_data.at[idx, feature] = test_avg_month[year_month]
   

In [9]:
print(train_data[rows].head())

          date  cabbage  radish   garlic    onion   daikon  cilantro  \
31  2021-02-07   553.20  356.68  4415.75  1030.00  3772.44   4243.76   
223 2021-09-20  2985.97  576.81  6959.62   893.28  1441.37   1706.55   
336 2022-02-03  1668.21  435.42  6796.20   450.00  1904.38   4907.77   
411 2022-05-01  2429.81  918.75  8100.00   635.36  1295.08   5041.66   
424 2022-05-15  2429.81  918.75  5440.00   635.36  1295.08   5041.66   

     artichoke year_month  
31     7213.89    2021-02  
223    5121.82    2021-09  
336   11550.61    2022-02  
411    6634.13    2022-05  
424    6634.13    2022-05  


In [10]:
train_data.drop(columns='year_month', inplace=True)
test_data.drop(columns='year_month', inplace=True)

In [11]:
print(train_data.isnull().sum())
print('=' * 50)
print(test_data.isnull().sum())

date         0
cabbage      0
radish       0
garlic       0
onion        0
daikon       0
cilantro     0
artichoke    0
dtype: int64
date         0
cabbage      0
radish       0
garlic       0
onion        0
daikon       0
cilantro     0
artichoke    0
dtype: int64
